# Run sandbox notebook

In [1]:
%run sandbox.ipynb

Current working directory: /home/darrenyhuang/projects/pleco-anki-server/src/resources
Loading credentials from token.json
Refreshing expired credentials
Failed to refresh credentials: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})
Authenticating with Google Drive
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=624862442234-4iconv0m99pu1luia7lub9c8m7t8of0b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A45942%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=SD38PdBTvlmJgwJMDmIIVGUktkroIy&access_type=offline


In [3]:
main()

Latest flashcard xml last modified time: 2025-01-02 12:17:32 EST-0500
1648 flashcard entries found| 1 error entries found


# Helper Methods

In [4]:
to_drop = [1030, 1118]

def drop(arr, to_drop):
    arr = arr.copy()
    for i in to_drop:
        arr.pop(i)
    return arr

In [5]:
import json

with open('flashcard_entries.json', 'r') as file:
    flashcard_entries = json.load(file)

to_del = []
for i, entry in enumerate(flashcard_entries):
    pinyin = ''.join(entry['pinyin'])
    anki_pinyin = entry['anki_pinyin']
    if pinyin != anki_pinyin:
        # print(i, entry['traditional'], pinyin, anki_pinyin)
        to_del.append(i)
for i in reversed(to_del):
    del flashcard_entries[i]
        


In [149]:
from utils.pinyin import *
from utils.html import *

In [7]:


# Example usage
file_path = "cedict_ts.u8"  # Path to the CC-CEDICT file
fifth_tone_pinyins = extract_fifth_tone_pinyin(file_path)
# print("Pinyin with the 5th tone:", fifth_tone_pinyins)

toneless_pinyin_set = extract_toneless_pinyin(file_path)
toneless_pinyin_trie = create_trie_from_pinyin(toneless_pinyin_set)


In [8]:
"ma" in fifth_tone_pinyins

True

In [9]:
import regex as re
import regex
import functools
import csv
import os

### Load CC-CEDICT (Only Traditional Variants) ###
@functools.lru_cache(maxsize=None)  # Infinite cache size
def load_cc_cedict(filename="cedict_ts.u8"):
    """Parses CC-CEDICT to extract traditional-only variant mappings."""
    variants = {}
    with open(filename, encoding="utf-8") as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue
            match = re.match(r"(\S+) (\S+) \[.*?\] /(.*?)/", line)
            if match:
                trad, simp, definition = match.groups()

                # Extract explicit variants from definitions: "/variant of X|Y[pinyin]/"
                variant_match = re.search(r"variant of ([\u4E00-\u9FFF\|]+)", definition)
                if variant_match:
                    var = variant_match.group(1).split("|")[0]
                    # Ensure bidirectional mapping
                    variants.setdefault(trad, set()).add(var)
                    variants.setdefault(var, set()).add(trad)

    return variants

### Load CC-CEDICT (Only Traditional Variants) ###
@functools.lru_cache(maxsize=None)  # Infinite cache size
def load_moedict(filename="moedict.csv"):
    variants = {}

    # Open the moedict.csv file
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Iterate through each row in the CSV
        for row in reader:
            term = row['字詞名']
            definition = row['釋義']
            
            # Search for variants indicated by 也作「<VARIANT>」
            matches = re.findall(r'也作「(.*?)」', definition)
            all_words = set([term] + list(matches))
            
            # Print the term and its variants
            for variant in all_words:
                variants.setdefault(variant, set()).update(all_words.difference({variant}))
    
    return variants

@functools.lru_cache(maxsize=None)  # Infinite cache size
def get_c_variants(folder_path="c"):
    """
    Iterates through all files in the specified folder and prints their contents.

    Args:
        folder_path (str): Path to the folder containing files.
    """
    global test_txt
    variants = {}
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            # Print the contents of the JSON file
            # data_json = json.dumps(data, indent=4, ensure_ascii=False)
            if file_name.startswith("@") or file_name.startswith("=") or file_name.startswith("xref"):
                continue
            non_chinese_or_bracket = regex.compile(r'[^「」\p{Han}]')
            word = regex.sub(non_chinese_or_bracket, "", data["t"])
            definitions = [
                regex.sub(non_chinese_or_bracket, "", d["f"])
                for h in data.get('h', [])  # Start from the 'h' key
                for d in h.get('d', [])  # Look inside the 'd' list
            ]
            for definition in definitions:
                # Search for variants indicated by 也作「<VARIANT>」
                matches = re.findall(r'也作「(.*?)」', definition)
                all_words = set([word] + list(matches))
                
                # Print the term and its variants
                if matches:
                    for variant in all_words:
                        variants.setdefault(variant, set()).update(all_words.difference({variant}))
    
    return variants

@functools.lru_cache(maxsize=None)  # Infinite cache size
def load_unihan_variants(filename="Unihan_Variants.txt"):
    """Parses Unihan_Variants.txt for character-level variants (traditional-only)."""
    variants = {}
    with open(filename, encoding="utf-8") as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue
            parts = line.split("\t")
            if len(parts) >= 3:
                char = chr(int(parts[0][2:], 16))  # Convert U+XXXX to character
                key = parts[1]
                
                # Extract only the U+XXXX part before '<' (if present)
                var_code = parts[2].split("<")[0].strip()
                
                try:
                    var = chr(int(var_code[2:], 16))  # Convert U+XXXX to character
                    
                    # Only keep traditional variants (ignore kSimplifiedVariant)
                    if "Variant" in key and "Simplified" not in key and "Traditional" not in key:
                        variants.setdefault(char, set()).add(var)
                        variants.setdefault(var, set()).add(char)
                except ValueError:
                    pass  # Skip bad format entries
    
    return variants


@functools.lru_cache(maxsize=None)  # Infinite cache size
def load_manual_variants(filename="manual_variants.csv"):
    """Parses manual_variants.csv to extract bidirectional variant mappings."""
    variants = {}
    
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        
        for row in reader:
            if len(row) < 2:
                continue  # Skip malformed rows
            
            words = set(word.strip() for word in row if word.strip())
            
            for word in words:
                variants.setdefault(word, set()).update(words.difference({word}))
    
    return variants


def get_variants(word):
    return load_cc_cedict().get(word, set()).union(load_moedict().get(word, set())).union(get_c_variants().get(word, set())).union(load_unihan_variants().get(word, set())).union(load_manual_variants().get(word, set()))


In [10]:
load_cc_cedict()
load_moedict()
get_c_variants()
load_unihan_variants()
load_manual_variants()
""

''

# label segs

In [11]:
import unicodedata

def fullwidth_to_ascii(text):
    return ''.join(
        unicodedata.normalize('NFKC', char) for char in text
    )

# # Example usage
# fullwidth_text = "ｉＯＳ 10"
# fullwidth_to_ascii(fullwidth_text)

In [12]:
def overlap_length(chinese: str, english: str) -> int:
    # Match all trailing non-Han characters from the Chinese string
    match = re.search(r'.*[\p{Han}]', chinese)
    if match:
        non_chinese_suffix = chinese[match.end():]  # Get trailing non-Han portion
    else:
        non_chinese_suffix = chinese  # Entire string is non-Han
    
    # Check how much of this suffix matches the start of the English string
    overlap = 0
    for i in range(1, len(non_chinese_suffix) + 1):
        if english.startswith(non_chinese_suffix[-i:]):
            overlap = i
    
    return overlap

In [13]:
chinese = "升級到 ｉＯＳ 10"
english = "ｉＯＳ 10 Upgrade to iOS 10."
print(overlap_length(chinese, english))  # Output: 6

6


In [17]:
import re
from collections import defaultdict

    
@functools.lru_cache(maxsize=None)  # Infinite cache size
def parse_cedict_toneless_pinyins(filename="cedict_ts.u8"):
    """Parse cedict_ts.u8 and extract a dictionary mapping Chinese characters to toneless Pinyin."""
    char_to_pinyin = defaultdict(set)
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            if line.startswith("#"):
                continue  # Skip comment lines
            
            parts = line.split()
            if len(parts) < 3:
                continue  # Skip malformed lines
            
            traditional, simplified, *pinyin_parts = parts
            
            pinyin_bracket_match = re.search(r'\[(.*?)\]', line)
            if not pinyin_bracket_match:
                continue
            
            pinyin_string = re.sub(r"[^A-Za-z ü]", "", pinyin_bracket_match.group(1).replace("u:", "ü"))
            pinyin_with_tones = pinyin_string.split()
            pinyin_toneless = [p for p in pinyin_with_tones]
            
            for char, pinyin_ in zip(traditional, pinyin_toneless):
                char_to_pinyin[char].add(pinyin_.lower())
                if pinyin_ == "-":
                    print(line)
    
    return char_to_pinyin

## variants

In [15]:
def update_example_sentence_with_variants(traditional_word, segment):
    """
    Updates example sentences in the segments list by checking if the traditional word or its variants
    are present in the example sentences. If a variant is found, it is added to the example sentence dict.

    Args:
        traditional_word (str): The traditional Chinese phrase to check.
        segments (list): List of segment dictionaries.

    Returns:
        list: Updated list of segments.
    """
    # Get the variants of the traditional word
    word_variants = get_variants(traditional_word)
    if segment['label'] == 'example_sentence':
        # Check if the traditional word is not in the Chinese segment
        if traditional_word not in segment['chinese']:
            # Search through the variants
            for variant in word_variants:
                if variant in segment['chinese']:
                    # Add the variant to the example sentence dict
                    segment['variant'] = variant
                    return True
    return False

def update_example_sentence_with_separated_words(traditional_word, segment, max_len=6):
    for word in [traditional_word] + list(get_variants(traditional_word)):
        if len(word) == 2 and segment['label'] == 'example_sentence' and word not in segment['chinese']:
            char1, char2 = word
            pattern = f"{char1}.{{0,{max_len}}}{char2}"
            match = re.search(pattern, segment['chinese'])
            # print(pattern, segment['chinese'], match)
            if not match:
                continue
            separated_words = match.group()
            segment['separated_word'] = separated_words
            return True
    return False

## split chinese pinyin example sentence util

In [102]:
import regex
import regex as re
from pypinyin import pinyin, Style

@functools.lru_cache(maxsize=None)  # Infinite cache size
def load_manual_pinyins(filename="manual_pinyins.csv"):
    """Parses manual_pinyins.csv to extract bidirectional variant mappings."""
    pinyins = {}
    
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        
        next(reader)  # Skip the header row
        for row in reader:
            if len(row) < 2:
                continue  # Skip malformed rows
            
            pinyins.setdefault(row[0], set()).add(row[1])  # add the pinyin to the set of pinyins for this character
    
    return pinyins


def strip_tone_marks(pinyin_with_tone):
    """
    Removes tone marks from pinyin to get the base pinyin (5th tone equivalent).
    Handles both lowercase and uppercase vowels with tone marks.
    
    Args:
        pinyin_with_tone (str): Pinyin with tone marks
        
    Returns:
        str: Pinyin without tone marks
    """
    # Map of vowels with tone marks to base vowels (lowercase)
    tone_marks_map_lower = {
        'ā': 'a', 'á': 'a', 'ǎ': 'a', 'à': 'a',
        'ē': 'e', 'é': 'e', 'ě': 'e', 'è': 'e',
        'ī': 'i', 'í': 'i', 'ǐ': 'i', 'ì': 'i',
        'ō': 'o', 'ó': 'o', 'ǒ': 'o', 'ò': 'o',
        'ū': 'u', 'ú': 'u', 'ǔ': 'u', 'ù': 'u',
        'ǖ': 'ü', 'ǘ': 'ü', 'ǚ': 'ü', 'ǜ': 'ü', 'ü': 'ü'
    }
    
    # Map of vowels with tone marks to base vowels (uppercase)
    tone_marks_map_upper = {
        'Ā': 'A', 'Á': 'A', 'Ǎ': 'A', 'À': 'A',
        'Ē': 'E', 'É': 'E', 'Ě': 'E', 'È': 'E',
        'Ī': 'I', 'Í': 'I', 'Ǐ': 'I', 'Ì': 'I',
        'Ō': 'O', 'Ó': 'O', 'Ǒ': 'O', 'Ò': 'O',
        'Ū': 'U', 'Ú': 'U', 'Ǔ': 'U', 'Ù': 'U',
        'Ǖ': 'Ü', 'Ǘ': 'Ü', 'Ǚ': 'Ü', 'Ǜ': 'Ü', 'Ü': 'Ü'
    }
    
    # Combine both maps
    tone_marks_map = {**tone_marks_map_lower, **tone_marks_map_upper}
    
    result = ''
    for char in pinyin_with_tone:
        result += tone_marks_map.get(char, char)
    
    return result

convert_punc_dict = {"。": ".", "！": "!", "？": "?", "，": ",", "；": ";", "：": ":", "《": "«", "》": "»"}
def convert_punc(char):
    return convert_punc_dict.get(char, char)

skippable_leftover_pinyin = [".", ","]

In [ ]:
def split_chinese_string(text):
    pattern = re.compile(r"\([^\(\)]*\)")
    parts = []
    i = 0

    for match in pattern.finditer(text):
        start, end = match.span()

        # Add characters before the match, one by one
        while i < start:
            parts.append(text[i])
            i += 1

        # Add the whole parenthetical as one item
        parts.append(match.group())
        i = end

    # Add any remaining characters after the last match
    while i < len(text):
        parts.append(text[i])
        i += 1

    return parts


def split_chinese_pinyin(example_sentence, trad_word=None, print_debug=False):
    try:
        return split_chinese_pinyin_helper(example_sentence, rmv_paren=False, trad_word=trad_word, print_debug=print_debug)
    except ValueError:
        return split_chinese_pinyin_helper(example_sentence, rmv_paren=True, trad_word=trad_word, print_debug=print_debug)

def split_chinese_pinyin_helper(example_sentence, rmv_paren, trad_word=None, print_debug=False):
    """
    Splits a Chinese string and its corresponding pinyin string into matching lists.
    Uses the regex package with {Han} pattern for accurate Chinese character detection.
    Handles both toned and tone-less pinyin matching, in both lowercase and uppercase.
    
    Args:
        example_sentence (dict): Dictionary containing 'chinese' and 'pinyin' keys
        rmv_paren (bool): Whether to remove parenthetical text from Chinese string
        trad_word (str, optional): Traditional word for debugging
        print_debug (bool, optional): Whether to print debug information
    
    Returns:
        dict: Updated example_sentence with 'chinese_list' and 'pinyin_list'
    
    Raises:
        ValueError: If the Chinese and pinyin strings cannot be properly aligned after trying all possibilities
    """
    if rmv_paren:
        # chinese_string = re.sub(r"\([^\(\)]*\)\s*", "", example_sentence['chinese']).rstrip()
        chinese_string = split_chinese_string(example_sentence['chinese'])
    else:
        chinese_string = example_sentence['chinese'].rstrip()
    pinyin_string = example_sentence['pinyin']

    # Check if the Chinese string contains any Han characters
    if not regex.search(r'\p{Han}', chinese_string):
        raise ValueError("The Chinese string contains no Han characters.")

    # Remove tones from the input pinyin string for matching
    toneless_pinyin_string = ''.join(strip_tone_marks(char) for char in pinyin_string)

    # Function to attempt matching with backtracking
    def backtrack_match():
        # Stack to keep track of state for backtracking
        # Each entry contains (position, chinese_list, pinyin_list, remaining_pinyin, remaining_toneless, pinyin_choices_idx)
        stack = [(0, [], [], pinyin_string.strip(), toneless_pinyin_string.strip(), 0, "")]
        
        while stack:
            i, current_chinese, current_pinyin, remain_py, remain_toneless_py, choice_idx, ignored = stack.pop()
            
            # Success condition: we've processed the entire Chinese string
            if i >= len(chinese_string):
                # Check if there's significant remaining pinyin
                if remain_py.strip() and remain_py.strip() not in skippable_leftover_pinyin:
                    # Add remaining pinyin to English and update the final result
                    if print_debug:
                        print(remain_py.strip(), trad_word, current_chinese, current_pinyin)
                    example_sentence['english'] = (remain_py + ' ' + example_sentence['english']).lstrip()
                    example_sentence['pinyin'] = example_sentence['pinyin'][:-len(remain_py)].rstrip()
                    if current_pinyin:
                        current_pinyin[-1] = current_pinyin[-1].strip()
                
                # We found a complete match
                return current_chinese, current_pinyin, ignored
            
            current_char = chinese_string[i]
            
            # Check if current character is Chinese using \p{Han} pattern
            if regex.match(r'\p{Han}', current_char):
                # Get all possible pinyins for this character
                possible_pinyins = list(set([p[0] for p in pinyin([current_char], style=Style.TONE, heteronym=True)]).union(
                    load_manual_pinyins().get(current_char, set())).union(
                    parse_cedict_toneless_pinyins().get(current_char, set())))
                
                # Add tone-less versions of each pinyin
                toneless_pinyins = sorted([strip_tone_marks(p) for p in possible_pinyins], key=len, reverse=True)
                
                # Try pinyin choices starting from choice_idx
                match_found = False
                for idx in range(choice_idx, len(toneless_pinyins)):
                    possible_toneless_pinyin = toneless_pinyins[idx]
                    
                    # Case-insensitive matching with tone-less pinyin
                    if remain_toneless_py.lower().startswith(possible_toneless_pinyin.lower()):
                        # Check for whitespace in the original pinyin string
                        whitespace_match = regex.match(
                            r'^' + regex.escape(possible_toneless_pinyin) + r'(\s*)', 
                            remain_toneless_py, 
                            regex.IGNORECASE
                        )
                        
                        if whitespace_match:
                            toneless_pinyin_with_space = whitespace_match.group(0)
                            
                            # Create new state with this match
                            new_chinese = current_chinese + [current_char]
                            new_pinyin = current_pinyin + [remain_py[:len(toneless_pinyin_with_space)]]
                            new_remain_py = remain_py[len(toneless_pinyin_with_space):]
                            new_remain_toneless = remain_toneless_py[len(toneless_pinyin_with_space):]
                            
                            # Push the current state for backtracking (in case this path fails)
                            # We'll try the next pinyin choice if we come back to this state
                            if idx + 1 < len(toneless_pinyins):
                                stack.append((i, current_chinese, current_pinyin, remain_py, remain_toneless_py, idx + 1, ignored))
                            
                            # Push the new state to continue with this match
                            stack.append((i + 1, new_chinese, new_pinyin, new_remain_py, new_remain_toneless, 0, ignored))
                            match_found = True
                            break
                
                # If no match found and we have remaining pinyin, try skipping one character from pinyin
                if not match_found and len(remain_py) > 0:
                    new_ignored = ignored + remain_py[0]
                    stack.append((i, current_chinese, current_pinyin, remain_py[1:], remain_toneless_py[1:], 0, new_ignored))
                # If no match found and no remaining pinyin, we need to continue exploring other paths
                
            elif current_char in convert_punc_dict and remain_toneless_py.startswith(convert_punc_dict.get(current_char)):
                # Handle punctuation
                whitespace_match = regex.match(
                    r'^' + regex.escape(convert_punc_dict.get(current_char)) + r'(\s*)', 
                    remain_toneless_py, 
                    regex.IGNORECASE
                )
                
                if whitespace_match:
                    toneless_pinyin_with_space = whitespace_match.group(0)
                    
                    new_chinese = current_chinese + [current_char]
                    new_pinyin = current_pinyin + [remain_py[:len(toneless_pinyin_with_space)]]
                    new_remain_py = remain_py[len(toneless_pinyin_with_space):]
                    new_remain_toneless = remain_toneless_py[len(toneless_pinyin_with_space):]
                    
                    stack.append((i + 1, new_chinese, new_pinyin, new_remain_py, new_remain_toneless, 0, ignored))
                elif len(remain_py) > 0:  # Only try ignoring if we have characters left
                    # If we can't match punctuation, try ignoring one character from pinyin
                    new_ignored = ignored + remain_py[0]
                    stack.append((i, current_chinese, current_pinyin, remain_py[1:], remain_toneless_py[1:], 0, new_ignored))
                    
            else:
                # For non-Chinese characters, process the segment
                non_chinese_segment = ""
                current_pos = i
                while current_pos < len(chinese_string) and not regex.match(r'\p{Han}', chinese_string[current_pos]):
                    non_chinese_segment += chinese_string[current_pos]
                    current_pos += 1
                
                # Add the non-Chinese segment to the lists
                if non_chinese_segment:
                    if current_chinese:  # If we have a current Chinese segment
                        # Check for whitespace at the start of the non-Chinese segment
                        whitespace_match = regex.match(r'^\s+', non_chinese_segment)
                        if whitespace_match: # Handle leading whitespace
                            whitespace = whitespace_match.group(0)
                            non_chinese_segment = non_chinese_segment[len(whitespace):]
                            # current_chinese[-1] += whitespace

                    if non_chinese_segment:
                        new_chinese = current_chinese + [non_chinese_segment]
                        
                        # Try to match and remove the non-Chinese segment from the pinyin string
                        if remain_py.startswith(non_chinese_segment):
                            new_pinyin = current_pinyin + [non_chinese_segment]
                            new_remain_py = remain_py[len(non_chinese_segment):]
                            new_remain_toneless = remain_toneless_py[len(non_chinese_segment):]
                        else:
                            # Handle case where non-Chinese characters might not appear in pinyin
                            new_pinyin = current_pinyin + [non_chinese_segment]
                            new_remain_py = remain_py
                            new_remain_toneless = remain_toneless_py
                    
                    stack.append((current_pos, new_chinese, new_pinyin, new_remain_py, new_remain_toneless, 0, ignored))
        
        # If we've exhausted all possibilities without finding a match
        if print_debug:
            if trad_word:
                print("Trad word:", trad_word)
            print("Chinese string:", chinese_string)
            print("Pinyin string:", pinyin_string)
                
        raise ValueError(f"Could not match pinyin for Chinese text '{chinese_string}' after trying all possibilities")
    
    # Run the backtracking algorithm
    try:
        chinese_list, pinyin_list, ignored_pinyin = backtrack_match()
        
        example_sentence['chinese_list'] = chinese_list
        example_sentence['pinyin_list'] = pinyin_list
        example_sentence['ignored_pinyin'] = ignored_pinyin.strip()
        
        return example_sentence
        
    except ValueError as e:
        if print_debug:
            print(f"Error: {e}")
        raise

In [68]:
load_manual_pinyins.cache_clear()

## separate out bold segments for chinese and pinyin

In [438]:
def add_bold_segments(example_sentence, traditional_word):
    split_chinese_pinyin(example_sentence)
    update_example_sentence_with_variants(traditional_word, example_sentence)
    update_example_sentence_with_separated_words(traditional_word, example_sentence)
    to_bold = None
    if traditional_word in example_sentence['chinese']:
        to_bold = traditional_word
    elif 'variant' in example_sentence:
        to_bold = example_sentence['variant']
    elif 'separated_word' in example_sentence:
        to_bold = example_sentence['separated_word']
    else:
        raise ValueError(f"No valid word found to bold for {traditional_word}, example sentence: {example_sentence}")
    
    chinese_final_list = []  # [{'segment': '升級到', 'bold': false}, ...]}
    pinyin_final_list = []  # [{'segment': 'shengjidao', 'bold': false}, ...]}
    
    # Collect characters and pinyins to process
    chinese_chars = example_sentence['chinese_list']
    pinyin_chars = example_sentence['pinyin_list']
    found_bold = False
    
    i = 0
    while i < len(chinese_chars):
        # Check if the current position starts a matching segment for to_bold
        if i + len(to_bold) <= len(chinese_chars) and ''.join(chinese_chars[i:i+len(to_bold)]) == to_bold:
            # Add the bold segment as a whole
            chinese_final_list.append({'segment': to_bold, 'bold': True})
            
            # Collect corresponding pinyin for the bold segment
            bold_pinyin = ''.join(pinyin_chars[i:i+len(to_bold)])
            bold_pinyin_whtspc = bold_pinyin[len(bold_pinyin.rstrip()):]
            bold_pinyin = bold_pinyin.rstrip()
            pinyin_final_list.append({'segment': bold_pinyin, 'bold': True})
            if bold_pinyin_whtspc:
                pinyin_chars[i+len(to_bold)] = bold_pinyin_whtspc + pinyin_chars[i+len(to_bold)]
            
            # Skip ahead past the bold segment
            found_bold = True
            i += len(to_bold)
        else:
            # Add non-bold character
            chinese_final_list.append({'segment': chinese_chars[i], 'bold': False})
            pinyin_final_list.append({'segment': pinyin_chars[i], 'bold': False})
            i += 1

    if not found_bold:
        raise ValueError(f"Could not find the word '{to_bold}' in the example sentence. {example_sentence}")
    
    # Now combine any adjacent segments with the same bold status
    # (This is technically redundant with the logic above but included for clarity)
    combined_chinese = []
    combined_pinyin = []
    
    if chinese_final_list:
        current_chinese = chinese_final_list[0]
        current_pinyin = pinyin_final_list[0]
        
        for i in range(1, len(chinese_final_list)):
            if chinese_final_list[i]['bold'] == current_chinese['bold']:
                # Combine with previous segment of same type
                current_chinese['segment'] += chinese_final_list[i]['segment']
                current_pinyin['segment'] += pinyin_final_list[i]['segment']
            else:
                # Add the completed segment and start a new one
                combined_chinese.append(current_chinese)
                combined_pinyin.append(current_pinyin)
                current_chinese = chinese_final_list[i]
                current_pinyin = pinyin_final_list[i]
        
        # Add the last segment
        combined_chinese.append(current_chinese)
        combined_pinyin.append(current_pinyin)
    
    example_sentence['chinese_list_w_bold_labels'] = combined_chinese
    example_sentence['pinyin_list_w_bold_labels'] = combined_pinyin

## label segments

In [ ]:
# segment definition

import regex as re
import json

# Load the part of speech keywords from the JSON file
with open("part_of_speech_keywords.json", "r") as file:
    part_of_speech_keywords = json.load(file)


def label_segments(text, traditional_word):
    segments = []
    part_of_speech_pattern = re.compile(
        r"\b("
        + "|".join(re.escape(keyword) for keyword in part_of_speech_keywords)
        + r")\b",
        re.IGNORECASE,
    )
    chinese_pattern = re.compile(r"[^\s\(\)\[\]]*\p{Han}+[^\s\(\)\[\]]*")
    pinyin_pattern = re.compile(
        r"\S*[āēīōūǖĀĒĪŌŪǕáéíóúǘÁÉÍÓÚǗǎěǐǒǔǚǍĚǏǑǓǙàèìòùǜÀÈÌÒÙǛ]\S*"
    )
    english_brackets_pattern = re.compile(r"\[[^\[\]]*\]")
    english_paren_pattern = re.compile(r"\([^\(\)]*\)")
    pleco_uead_pattern = re.compile(r"\uead1.*?\uead2")

    pos_matches = list(part_of_speech_pattern.finditer(text))
    uead_matches = list(pleco_uead_pattern.finditer(text))
    chinese_matches = list(chinese_pattern.finditer(text))
    pinyin_matches = list(pinyin_pattern.finditer(text))
    english_brackets_matches = list(english_brackets_pattern.finditer(text))
    english_paren_pattern = list(english_paren_pattern.finditer(text))

    all_matches = sorted(
        pos_matches
        + english_brackets_matches
        + english_paren_pattern
        + uead_matches
        + chinese_matches
        + pinyin_matches,
        key=lambda x: x.start(),
    )

    last_end = 0
    for match in all_matches:
        if match.start() < last_end:
            continue
        if match.start() > last_end:
            segments.append(
                {"segment": text[last_end : match.start()], "label": "english"}
            )

        if match in pos_matches:
            segments.append(
                {"segment": match.group(), "label": "temp part of speech"}
            )
        elif match in english_brackets_matches:
            segments.append({"segment": match.group(), "label": "english"})
        elif match in english_paren_pattern:
            target_str = match.group()
            if re.match(pinyin_pattern, target_str):
                segments.append({"segment": target_str, "label": "pinyin"})
            elif re.match(r"^[\p{Han}《》=]+$", re.sub(r"[\(\)\s]", "", target_str)):
                segments.append({"segment": target_str, "label": "chinese"})
            else:
                segments.append({"segment": target_str, "label": "english"})
        elif match in uead_matches:
            segments.append({"segment": match.group(), "label": "english"})
        elif match in chinese_matches:
            segments.append({"segment": match.group(), "label": "chinese"})
        elif match in pinyin_matches:
            segments.append({"segment": match.group(), "label": "pinyin"})
        last_end = match.end()

    if last_end < len(text):
        segments.append({"segment": text[last_end:], "label": "english"})

    # process empty segments and whitespace
    segments = filter_empty_segs(segments)
    segments = process_whitespace_english(segments)
    segments = combine_adjacent_segments(segments)
    segments = shift_leading_whitespace(segments)

    # adjust pinyin for toneless pinyin
    segments = process_fifth_tone_pinyin(segments)
    segments = filter_empty_segs(segments)
    segments = combine_adjacent_segments(segments)
    segments = shift_leading_whitespace(segments)

    # process item numbers
    segments = process_item_numbers(segments)
    segments = filter_empty_segs(segments)
    segments = combine_adjacent_segments(segments)
    # print(segments)

    # process parts of speech
    segments = process_parts_of_speech(segments)
    segments = filter_empty_segs(segments)
    segments = combine_adjacent_segments(segments)

    segments = combine_pinyin_english_pinyin(segments)
    segments = combine_adjacent_segments(segments)
    segments = combine_example_sentences(segments)
    segments = combine_adjacent_segments(segments, {("english", "chinese"): "english"})
    segments = bold_example_sentences(segments, traditional_word)
    segments = combine_adjacent_segments(segments)

    segments = convert_segment_labels(segments, "chinese", "english")
    segments = convert_segment_labels(segments, "pinyin", "english")
    segments = combine_adjacent_segments(segments)

    return segments

def convert_segment_labels(segments, from_label, to_label):
    new_segments = []
    for seg in segments:
        if seg["label"] == from_label:
            seg["label"] = to_label
        new_segments.append(seg)
    return new_segments
    

def process_parts_of_speech(segments):
    new_segments = []
    i = 0

    # process POS at the very beginning
    while i < len(segments):
        if segments[i]["label"] == "temp part of speech":
            segments[i]["label"] = "part of speech"
            new_segments.append(segments[i])
            i += 1
        else:
            break
    
    last_seg = None
    # for every segment where the last segment is a line number, convert POS
    while i < len(segments):
        if segments[i]["label"] == "temp part of speech":
            if last_seg and last_seg['label'] in ["item_number", "part of speech"]:
                segments[i]["label"] = "part of speech"
            elif last_seg and last_seg['label'] == "english" and last_seg['segment'][-1] == "\n":
                segments[i]["label"] = "part of speech"
            else:
                segments[i]["label"] = "english"

        last_seg = segments[i]
        new_segments.append(segments[i])
        i += 1
    return new_segments


def shift_leading_whitespace(segments):
    for i in range(1, len(segments)):
        current_segment = segments[i]['segment']
        match = re.match(r'^(\s+)', current_segment)
        if match:
            leading_ws = match.group(1)
            # Remove leading whitespace from current segment
            segments[i]['segment'] = current_segment[len(leading_ws):]
            # Append it to the previous segment
            segments[i - 1]['segment'] += leading_ws
    return segments

def process_whitespace_english(segments):
    new_segments = []
    last_seg = None
    for seg in segments:
        if last_seg and seg["label"] == "english" and re.fullmatch(r'\s*', seg["segment"]) is not None:
            last_seg["segment"] += seg["segment"]
        else:
            new_segments.append(seg)
            last_seg = seg
    return new_segments

def bold_example_sentences(segments, traditional_word):
    new_segments = []
    for seg in segments:
        if seg["label"] == "example_sentence":
            try:
                add_bold_segments(seg, traditional_word=traditional_word)
            except ValueError as e:
                print(f"{traditional_word}: Error processing example sentence: {seg}, converting to english")
                # update_example_sentence_with_variants(traditional_word, seg)
                # update_example_sentence_with_separated_words(traditional_word, seg)
                seg = {
                    "segment": seg["chinese"] + seg["pinyin"] + seg["english"],
                    "label": "english",
                }
            new_segments.append(seg)
        else:
            new_segments.append(seg)
    return new_segments


def update_example_sentence_english_chinese_overlap(segment):
    if segment["label"] == "example_sentence":
        chinese = segment["chinese"]
        english = segment["english"]
        overlap = overlap_length(chinese, english)
        if overlap > 0:
            segment["english"] = english[overlap:]
            segment["pinyin"] += " " + english[:overlap]
            # print(chinese, english, segment)
    return segment


def combine_example_sentences(segments):
    new_segments = []
    i = 0
    while i + 2 < len(segments):
        if (
            segments[i]["label"] == "chinese"
            and segments[i + 1]["label"] == "pinyin"
            and segments[i + 2]["label"] == "english"
        ):
            combined_segment = {
                "label": "example_sentence",
                "chinese": segments[i]["segment"],
                "pinyin": segments[i + 1]["segment"],
                "english": segments[i + 2]["segment"],
            }
            combined_segment = update_example_sentence_english_chinese_overlap(combined_segment)
            new_segments.append(combined_segment)
            i += 3
        elif (  # special case
            i + 3 < len(segments)
            and segments[i]["label"] == "chinese"
            and segments[i + 1]["label"] == "english"
            # and len(segments[i + 1]["segment"]) <= 1
            and segments[i + 2]["label"] == "pinyin"
            and segments[i + 3]["label"] == "english"
        ):
            combined_segment = {
                "label": "example_sentence",
                "chinese": segments[i]["segment"],
                "pinyin": segments[i + 2]["segment"],
                "english": segments[i + 3]["segment"],
            }
            extra_segment = segments[i + 1]["segment"]
            if extra_segment == "。":
                combined_segment["chinese"] += extra_segment
            elif combined_segment["chinese"].startswith(extra_segment):
                combined_segment["pinyin"] = extra_segment + " " + combined_segment["pinyin"]
            else:
                combined_segment["chinese"] += " " + extra_segment
            combined_segment = update_example_sentence_english_chinese_overlap(combined_segment)

            # print("special case", combined_segment)
            # print(segments[i + 1])
            new_segments.append(combined_segment)
            i += 4
        else:
            new_segments.append(segments[i])
            i += 1
    new_segments.extend(segments[i:])
    return new_segments


def process_item_numbers(segments):
    # Process sequences between "part of speech" segments
    def search(segment_str, num, is_one=False):
        regex_patt = str(num) + r"(?=($|\s))"
        if is_one:
            regex_patt = r"(^|(\(-//-\) )|(\uead2 ))" + regex_patt
        else:
            regex_patt = r"(?<=(^|\s))" + regex_patt
        return re.search(regex_patt, segment_str)

    new_segments = []
    i = 0
    num = 1
    while i < len(segments):
        seg = segments[i]
        if (
            seg["label"] == "english"
            and (search(seg["segment"], 1, is_one=True) or (num != 1 and search(seg["segment"], num)))
        ):
            if search(seg["segment"], 1, is_one=True):
                num = 1
            
            start_index = search(seg["segment"], num).start()
            new_segments.append(
                {
                    "segment": seg["segment"][:start_index],
                    "label": "english",
                }
            )
            new_segments.append(
                {"segment": str(num), "label": "item_number"}
            )
            seg["segment"] = seg["segment"][
                start_index + len(str(num)) :
            ].lstrip()
            num += 1
            continue  # don't update i
        else:
            new_segments.append(seg)
            i += 1
    return new_segments


def filter_empty_segs(segments):
    return [segment for segment in segments if segment["segment"]]

def combine_adjacent_segments(segments, equivalent_labels=None):
    if equivalent_labels is None:
        equivalent_labels = {}
    else:
        equivalent_labels = {
            tuple(sorted(list(k))): v for k, v in equivalent_labels.items()
        }

    combined_segments = []
    for segment in segments:
        if combined_segments and segment["label"] not in ["example_sentence", "part of speech", "temp part of speech"]:
        # if combined_segments and segment["label"] not in ["example_sentence"]:
            last_label = combined_segments[-1]["label"]
            current_label = segment["label"]
            eq_label = tuple(sorted([last_label, current_label]))
            if last_label == current_label or eq_label in equivalent_labels:
                # combined_segments[-1]["segment"] += " " + segment["segment"]
                combined_segments[-1]["segment"] += segment["segment"]
                if eq_label in equivalent_labels:
                    combined_segments[-1]["label"] = equivalent_labels[eq_label]
            else:
                combined_segments.append(segment)
        else:
            combined_segments.append(segment)
    return combined_segments


def process_fifth_tone_pinyin(segments):
    new_segments = []
    for i in range(len(segments) - 1):
        current_segment = segments[i]
        next_segment = segments[i + 1]
        new_segments.append(current_segment)

        done = False
        while not done:
            if (
                current_segment["label"] in ["chinese", "pinyin"]
                and next_segment["label"] == "english"
            ):
                for pinyin in fifth_tone_pinyins:
                    regex_patt = "^" + pinyin + r"($|[^a-zA-Z])"
                    mtch = re.match(regex_patt, next_segment["segment"].lower())
                    if mtch:
                        pinyin_seg, rest = (
                            next_segment["segment"][: mtch.end()],
                            next_segment["segment"][mtch.end() :],
                        )

                        new_segments.append(
                            # {"segment": pinyin_seg.strip(), "label": "pinyin"}
                            {"segment": pinyin_seg, "label": "pinyin"}
                        )
                        # next_segment["segment"] = rest.strip()
                        next_segment["segment"] = rest
                        break
                else:
                    done = True
            else:
                done = True
    new_segments.append(segments[-1])
    return new_segments


def combine_pinyin_english_pinyin(segments):
    new_segments = []
    segments = segments.copy()
    i = 0
    while i + 2 < len(segments):
        if (
            segments[i]["label"] == "pinyin"
            and segments[i + 1]["label"] == "english"
            and segments[i + 2]["label"] == "pinyin"
        ):
            combined_segment = {
                "segment": segments[i]["segment"] + segments[i + 1]["segment"] + segments[i + 2]["segment"],
                # "segment": segments[i]["segment"] + " " + segments[i + 1]["segment"] + " " + segments[i + 2]["segment"],
                "label": "pinyin",
            }
            i += 2
            segments[i] = combined_segment
        else:
            new_segments.append(segments[i])
            i += 1
    new_segments.extend(segments[i:])
    return new_segments

# test bold

In [296]:
for entry in flashcard_entries:
    if entry['traditional'] == '反正':
        segments = label_segments(entry['definition'], entry['traditional'])

# Format Entries

In [349]:
def fmt_entry(entry):
    traditional = entry.get("traditional", "")
    simplified = entry.get("simplified", "")
    pinyin = entry.get("pinyin", "")
    definition = entry.get("definition", "")
    simplified_hint = f"〔{simplified}〕" if traditional != simplified else ""

    formatted_back = ""
    formatted_back += f'<div align="left"><p><span style="font-size:32px">{traditional}{simplified_hint}</span><br/>\n'
    formatted_back += '<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span>'

    # pinyin
    for p in pinyin:
        starters = ["//", " ", "-", "→"]
        while any([p.startswith(s) for s in starters]):
            for s in starters:
                if p.startswith(s):
                    formatted_back += f'<span style="font-weight:600;">{s}</span>'
                    p = p.replace(s, "", 1)
        formatted_back += f'<span style="color:{get_pinyin_color(p)};"><span style="font-weight:600;">{p}</span></span>'
    
    # part of speech
    formatted_back += '</p>\n</div><div align="left"><p>'

    last_label = None
    for segment in label_segments(definition, traditional_word=traditional):
        if segment["label"] == "part of speech":
            if last_label == "example_sentence":
                formatted_back += '<br/>\n</p>\n</blockquote>\n<p><br/>\n'
            elif last_label == "part of speech":
                formatted_back += '<br/>\n'
            elif last_label == "english":
                formatted_back += '</p>\n<p>'
            formatted_back += f'<b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">{segment["segment"].upper().strip()}</span></span></b>'


        elif segment["label"] == "english":
            # process transition
            if last_label == "part of speech":
                formatted_back += '<br/>\n'
            # add english segmewnt
            formatted_back += segment["segment"].strip()


        elif segment["label"] == "example_sentence":
            # process transition
            if last_label == "english":
                formatted_back += '<br/>\n</p>\n'
            elif last_label == "example_sentence":
                formatted_back += '<br/>\n</p>\n</blockquote>\n'
            
            # process example sentence
            formatted_back += '<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p>'
            # chinese
            for ex_seg in segment["chinese_list_w_bold_labels"]:
                formatted_back += f'<span style="color:#0078C3;">'
                if ex_seg["bold"]:
                    formatted_back += f'<b>{ex_seg["segment"]}</b>'
                else:
                    formatted_back += ex_seg["segment"]
                formatted_back += '</span>'
            formatted_back += '<br/>\n'
            # pinyin
            for ex_seg in segment["pinyin_list_w_bold_labels"]:
                if ex_seg["bold"]:
                    formatted_back += f'<b>{ex_seg["segment"]}</b>'
                else:
                    formatted_back += f'<span style="font-weight:600;">{ex_seg["segment"]}</span>'
            formatted_back += '<br/>\n'
            # english
            formatted_back += segment["english"].strip()
            # formatted_back += '<br/>\n</p>\n</blockquote>'

        elif segment["label"] == "item_number":
            formatted_back += '<br/>\n'
            if last_label == "example_sentence":
                formatted_back += '</p>\n</blockquote>\n<p>'
            formatted_back += f'<b>{segment['segment'].strip()}\t</b>'
        # elif segment["label"] == "pinyin":
        #     formatted_back += f'<span style="color:{get_pinyin_color(segment["segment"])};"><b><span style="font-size:0.80em;">{segment["segment"]} </span></b></span>'
        # else:
        #     formatted_back += f'<span style="color:#000000;"><b><span style="font-size:0.80em;">{segment["segment"]} </span></b></span>'
        else:
            print(traditional, ":", segment)
        last_label = segment["label"]

    # final
    formatted_back += '</p>'
    if last_label == "example_sentence":
        formatted_back += '\n</blockquote>'

    formatted_back += '\n</div>'

    return formatted_back

# Grade Entries

In [226]:
class ToneColor(Enum):
    RED = "#E30000"
    GREEN = "#02B31C"
    PURPLE = "#8900BF"
    BLUE = "#1510F0"
    GREY = "#777777"  # Neutral tone


def get_pinyin_color(pinyin):
    tone_map = {
        "āēīōūǖĀĒĪŌŪǕ": ToneColor.RED,
        "áéíóúǘÁÉÍÓÚǗ": ToneColor.GREEN,
        "ǎěǐǒǔǚǍĚǏǑǓǙ": ToneColor.BLUE,
        "àèìòùǜÀÈÌÒÙǛ": ToneColor.PURPLE,
    }

    tone_color = ToneColor.GREY  # Default to neutral tone
    for chars, t in tone_map.items():
        if any(char in pinyin for char in chars):
            tone_color = t
            break

    return tone_color.value

In [261]:
from bs4 import BeautifulSoup


def reorder_bold_and_color_spans(html_text):
    # Parse the HTML text
    soup = BeautifulSoup(f"<root>{html_text}</root>", "html.parser")
    
    # Find all bold tags
    bold_tags = soup.find_all("b")
    
    for bold_tag in bold_tags:
        # Check if the bold tag contains a span with the specific color #0078C3
        color_span = bold_tag.find("span", style=lambda s: s and "color:#0078C3" in s)
        
        if color_span:
            # Get the content of the span
            content = color_span.contents
            
            # Create new structure: span outside, b inside
            new_span = soup.new_tag("span")
            new_span["style"] = color_span["style"]
            
            new_bold = soup.new_tag("b")
            new_bold.extend(content)
            
            new_span.append(new_bold)
            
            # Replace the old structure with the new one
            bold_tag.replace_with(new_span)
    
    # Convert the soup back to a string
    return str(soup).replace("<root>", "").replace("</root>", "")

In [400]:
def fix_separated_pos_tags(html_text):
    pos_text = '<span style="font-size:0.80em;"><span style="color:#B4B4B4;">.*?</span></span>'
    patt = f'({pos_text})</b> <b>({pos_text})'
    previous_text = ""
    current_text = html_text
    while current_text != previous_text:
        previous_text = current_text
        current_text = re.sub(patt, r'\1</b><br/>\n<b>\2', current_text)
    return current_text

In [398]:

def grade_fmt_entry(flashcard_entries, n_error_char_show=10, to_drop=[], stop_at_fail=False):
    flashcard_entries = drop(flashcard_entries, to_drop)
    correct_count = 0
    length_diff_count = 0
    wrong_count = 0
    for i, entry in enumerate(flashcard_entries):
        expected = entry['formatted_back']
        expected = expected.replace(' ;=""', ";")
        expected = reorder_nested_spans(expected)
        expected = reorder_bold_and_color_spans(expected)
        expected = re.sub(r"<plecoentry.*?</plecoentry>$", "", expected)
        expected = expected.replace("\xa0", " ")
        expected = fix_separated_pos_tags(expected)
        result = fmt_entry(entry)
        
        if expected != result:
            for j in range(min(len(expected), len(result))):
                if expected[j] != result[j]:
                    wrong_count += 1
                    if stop_at_fail:
                        print(f"Total correct entries: {correct_count}")
                        print(f"Total wrong entries: {wrong_count}")
                        print(f"Total entries differing only in length: {length_diff_count}")
                        print(f"Entry {i} differs at character {j}:")
                        print(f"Exp: {repr(expected[j:j+n_error_char_show])}...")
                        print(f"Got: {repr(result[j:j+n_error_char_show])}...")
                        print(f"Def: {entry['definition']}...")
                        print("wrd:", repr(entry['traditional']))
                        print("exp:", repr(expected))
                        print("res:", repr(result))
                        return
                    break
            else:
                length_diff_count += 1
                j = len(result)
                if stop_at_fail:
                    print(f"Total correct entries: {correct_count}")
                    print(f"Total wrong entries: {wrong_count}")
                    print(f"Total entries differing only in length: {length_diff_count}")
                    print(f"{i}: {repr(expected[j:j+n_error_char_show])}...")
                    print("wrd:", repr(entry['traditional']))
                    print("def:", repr(entry['definition']))
                    print("exp:", repr(expected))
                    print("res:", repr(result))
                    return
                continue
        else:
            correct_count += 1
    print(f"Total correct entries: {correct_count}")
    print(f"Total wrong entries: {wrong_count}")
    print(f"Total entries differing only in length: {length_diff_count}")

## actual grade run

In [436]:
get_variants("舖")

{'舗', '鋪'}

In [433]:
word = '舖'
entry = [entry for entry in flashcard_entries if entry['traditional'] == word][0]
label_segments(entry['definition'], traditional_word=entry['traditional'])
# entry['definition']
# ""

[{'segment': 'noun ', 'label': 'part of speech'},
 {'segment': 'shop; store ', 'label': 'english'},
 {'label': 'example_sentence',
  'chinese': '雜貨鋪兒 ',
  'pinyin': 'záhuòpùr ',
  'english': 'general store \n',
  'chinese_list': ['雜', '貨', '鋪', '兒'],
  'pinyin_list': ['zá', 'huò', 'pù', 'r'],
  'ignored_pinyin': '',
  'variant': '鋪',
  'chinese_list_w_bold_labels': [{'segment': '雜貨', 'bold': False},
   {'segment': '鋪', 'bold': True},
   {'segment': '兒', 'bold': False}],
  'pinyin_list_w_bold_labels': [{'segment': 'záhuò', 'bold': False},
   {'segment': 'pù', 'bold': True},
   {'segment': 'r', 'bold': False}]},
 {'segment': 'noun ', 'label': 'part of speech'},
 {'segment': 'plank bed ', 'label': 'english'},
 {'label': 'example_sentence',
  'chinese': '床鋪 ',
  'pinyin': 'chuángpù ',
  'english': 'bed \n',
  'chinese_list': ['床', '鋪'],
  'pinyin_list': ['chuáng', 'pù'],
  'ignored_pinyin': '',
  'variant': '鋪',
  'chinese_list_w_bold_labels': [{'segment': '床', 'bold': False},
   {'segment

In [434]:
word = '舖'
entry = [entry for entry in flashcard_entries if entry['traditional'] == word][0]
grade_fmt_entry([entry], 1000, to_drop=[])

Total correct entries: 1
Total wrong entries: 0
Total entries differing only in length: 0


In [435]:
len([entry for entry in flashcard_entries if entry['traditional'] == word])

1

In [440]:
grade_fmt_entry(flashcard_entries, 1000, to_drop=[])

令: Error processing example sentence: {'label': 'example_sentence', 'chinese': '(散曲) ', 'pinyin': 'song ', 'english': '(i.e. not in a sequence, corresponding to one aria in the drama) 1 [usu. in tune titles] '}, converting to english
Total correct entries: 1093
Total wrong entries: 511
Total entries differing only in length: 0


In [445]:
grade_fmt_entry(flashcard_entries, 1000, to_drop=[14, 21], stop_at_fail=True)
# grade_fmt_entry(flashcard_entries[6:], 1000)

Total correct entries: 21
Total wrong entries: 1
Total entries differing only in length: 0
Entry 21 differs at character 533:
Exp: '(一)</span><span style="color:#0078C3;"><b>瞬</b></span><span style="color:#0078C3;">間</span><br/>\n<b>shùn</b><span style="font-weight:600;">jiān</span><br/>\ninstant; in the twinkling of an eye; in a split second; in a flash</p>\n</blockquote>\n</div>'...
Got: '<b>瞬</b></span><span style="color:#0078C3;">間</span><br/>\n<b>shùn</b><span style="font-weight:600;">jiān</span><br/>\ninstant; in the twinkling of an eye; in a split second; in a flash</p>\n</blockquote>\n</div>'...
Def: verb wink; twinkle (一)瞬間 shùnjiān instant; in the twinkling of an eye; in a split second; in a flash...
wrd: '瞬'
exp: '<div align="left"><p><span style="font-size:32px">瞬</span><br/>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#8900BF;"><span style="font-weight:600;">shùn</span></span></p>\n</div><div align="left"><p><b><

In [162]:
fmt_entry(flashcard_entries[0])

'<div align="left"><p><span style="font-size:32px">艘</span><br/>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#E29999;"><span style="font-weight:600;">sōu</span></span></p>\n</div><div align="left"><p>'

# TODOS!
include removed () from the example sentence into the final example sentence output